<a href="https://colab.research.google.com/github/dakilaledesma/quick_morphology_explainer/blob/main/colab_notebooks/S2D_FullGrad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q drive/MyDrive/UNC/H2022/orchidaceae_test.zip -d /content/

CPU times: user 90.7 ms, sys: 23.3 ms, total: 114 ms
Wall time: 17 s


In [2]:
! pip install timm grad-cam tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 509 kB 4.2 MB/s 
     |████████████████████████████████| 23.0 MB 1.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.0-py3-none-any.whl size=28155 sha256=dd33dfb0864fa88ccbfdda791190a1fb6b89b15433a2f4807c7a9102028678ac
  Stored in directory: /root/.cache/pip/wheels/a5/dd/4c/8a6ae2754b8c2fa442c78ce0eee39c14dc8e21cb3d27604ec2
Successfully built grad-cam


In [3]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os, json
from glob import glob
from tqdm.notebook import tqdm

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

import timm

In [4]:
torch.backends.cudnn.enabled = False

In [5]:
def get_input_transform():
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])       
    transf = transforms.Compose([
        transforms.Resize((600, 600)),
        transforms.ToTensor(),
        normalize
    ])    

    return transf

def get_input_tensors(img):
    transf = get_input_transform()
    return transf(img).unsqueeze(0)

In [6]:
ckpt_fn = "drive/MyDrive/UNC/H2022/output/20220722-163041-sequencer2d_l-600/last.pth.tar"
im_path = "orchidaceae_test"
out_path = "resolution/600_s2d"
model_type = "sequencer2d_l"

fl_dict = {}
for idx, fl in enumerate(glob(f"{im_path}/*")):
    bn = os.path.basename(fl)
    fl_dict[bn] = idx
    
images = []
for im_fn in glob(f"{im_path}/**/*.*", recursive=True):
    bn = os.path.basename(im_fn)
    images.append(im_fn)
    
print(len(images))

4577


In [ ]:
model = timm.create_model(
        model_type,
        num_classes=300,
        in_chans=3,
        checkpoint_path=ckpt_fn)

In [ ]:
for im_fn in tqdm(images, total=len(list(images))):
    curr_dir = dir()
    cam_list = [["fullgrad", FullGrad(model=model, target_layers=[], use_cuda=True)]]
    
    bn = os.path.basename(im_fn)
    cat = fl_dict[im_fn.split("/")[-2]]
    
    img = Image.open(im_fn)
    img_t = get_input_tensors(img)
    test_img = np.array(img.resize((600, 600)), dtype='float32')
    test_img /= 255

    targets = [ClassifierOutputTarget(cat)]
    
    for cam_str, cam_method in cam_list:
        grayscale_cam = cam_method(input_tensor=img_t, targets=targets)
        grayscale_cam = grayscale_cam[0, :]

        visualization = show_cam_on_image(test_img, grayscale_cam, use_rgb=True)
        plt.imshow(visualization)
        if not os.path.exists(f"outs/{out_path}/{cam_str}/{bn}"):
            try:

                plt.savefig(f"outs/{out_path}/{cam_str}/{bn}")
            except FileNotFoundError:
                os.makedirs(f"outs/{out_path}/{cam_str}/")
            plt.close()
        
        del grayscale_cam
        
    for name in dir():
        if name not in curr_dir and name != "curr_dir":
            del globals()[name]

In [ ]:
! zip -q -r -0 s2d_outs.zip outs/

In [ ]:
! mv s2d_outs.zip /content/drive/MyDrive/UNC/S2022P/s2d_outs.zip